In [ ]:
!nvidia-smi

Sun Nov  7 15:48:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip -q install torchaudio
# !pip -q install datasets

import pandas as pd
import numpy as np
import os
from tqdm import tqdm_notebook as tqdm
from shutil import copyfile

import librosa
import librosa.display
import scipy.io.wavfile as wav

import IPython.display as ipd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

     |████████████████████████████████| 2.9 MB 5.3 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.3 MB/s eta 0:00:38tcmalloc: large alloc 1147494400 bytes == 0x55c75afea000 @  0x7f7fdd9ed615 0x55c7228764cc 0x55c72295647a 0x55c7228792ed 0x55c72296ae1d 0x55c7228ece99 0x55c7228e79ee 0x55c72287abda 0x55c7228ecd00 0x55c7228e79ee 0x55c72287abda 0x55c7228e9737 0x55c72296bc66 0x55c7228e8daf 0x55c72296bc66 0x55c7228e8daf 0x55c72296bc66 0x55c7228e8daf 0x55c72287b039 0x55c7228be409 0x55c722879c52 0x55c7228ecc25 0x55c7228e79ee 0x55c72287abda 0x55c7228e9737 0x55c7228e79ee 0x55c72287abda 0x55c7228e8915 0x55c72287aafa 0x55c7228e8c0d 0x55c7228e79ee
     |████████████████████████████████| 881.9 MB 20 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu111 requires torch==1.9.0, but you have torch 1.10.0 which is incompatible.
torchtext 0.10.0 re

In [ ]:
import pandas as pd
from torch.utils.data import Dataset

import torch

In [ ]:
!7z x /content/drive/MyDrive/Tesi/dataset_pat_bandPass2.zip -o/content/


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/Tesi/                                      1 file, 598041763 bytes (571 MiB)

Extracting archive: /content/drive/MyDrive/Tesi/dataset_pat_bandPass2.zip
--
Path = /content/drive/MyDrive/Tesi/dataset_pat_bandPass2.zip
Type = zip
Physical Size = 598041763

  0%      0% 4 - dataset_pat_bandPass2/132_2b2_Lr_mc_LittC2SE_bandPass2.wav                                                                     1% 12 - dataset_pat_bandPass2/203_1p3_Pr_mc_AKGC417L_bandPass2.wav

In [ ]:
train_pathology = pd.read_csv('/content/drive/MyDrive/Tesi/train_multiclass_pathology.csv')
test_pathology = pd.read_csv('/content/drive/MyDrive/Tesi/test_multiclass_pathology.csv')

In [ ]:
train_multiclass = train_pathology.copy()
train_multiclass

,filename,label
0,/content/dataset_pat_bandPass2/205_1b3_Ar_mc_A...,1
1,/content/dataset_pat_bandPass2/151_3p2_Al_mc_A...,1
2,/content/dataset_pat_bandPass2/226_1b1_Ll_sc_M...,2
3,/content/dataset_pat_bandPass2/130_3p2_Pl_mc_A...,1
4,/content/dataset_pat_bandPass2/135_2b2_Al_mc_L...,2
...,...,...
731,/content/dataset_pat_bandPass2/186_3b3_Tc_mc_A...,1
732,/content/dataset_pat_bandPass2/112_1p1_Pl_sc_L...,1
733,/content/dataset_pat_bandPass2/138_1p3_Pl_mc_A...,1
734,/content/dataset_pat_bandPass2/197_1b1_Al_sc_M...,2


In [ ]:
test_multiclass = test_pathology.copy()
test_multiclass

,filename,label
0,/content/dataset_pat_bandPass2/199_2b3_Ll_mc_L...,1
1,/content/dataset_pat_bandPass2/181_1b3_Tc_mc_L...,1
2,/content/dataset_pat_bandPass2/195_1b1_Al_sc_L...,1
3,/content/dataset_pat_bandPass2/160_1b3_Tc_mc_A...,1
4,/content/dataset_pat_bandPass2/211_1p2_Ar_mc_A...,1
...,...,...
179,/content/dataset_pat_bandPass2/139_1b1_Ll_sc_L...,1
180,/content/dataset_pat_bandPass2/186_3b3_Pl_mc_A...,1
181,/content/dataset_pat_bandPass2/163_2b2_Lr_mc_A...,1
182,/content/dataset_pat_bandPass2/217_1b1_Tc_sc_M...,0


In [ ]:
train_multiclass.groupby('label').count()

,filename
label,
0,25
1,648
2,63


In [ ]:
test_multiclass.groupby('label').count()

,filename
label,
0,10
1,162
2,12


In [ ]:
!pip -q install torchaudio
!pip install transformers

     |████████████████████████████████| 3.1 MB 5.5 MB/s 
     |████████████████████████████████| 895 kB 41.2 MB/s 
     |████████████████████████████████| 596 kB 41.3 MB/s 
     |████████████████████████████████| 59 kB 6.4 MB/s 
     |████████████████████████████████| 3.3 MB 30.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import librosa
total_duration=0
num_file=0
for elem,label in tqdm(zip(train_multiclass['filename'], train_multiclass['label']), total=train_multiclass.shape[0]):
  total_duration += librosa.get_duration(filename=elem)
  num_file = num_file+1

print(total_duration/num_file)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/736 [00:00<?, ?it/s]

21.596434425219353


In [ ]:
import torchaudio
duration = 20
freq = 16
class AudiosDataset(Dataset):
    def __init__(self, paths=None, targets=None, augment=False) -> None:
        self.paths = paths
        self.targets = targets
        self.augment = augment
        
    
    def __len__(self) -> int:
        return len(self.paths)

    def __getitem__(self, idx: int) -> dict:
        y, sr = torchaudio.load(self.paths.iloc[idx])#, normalization=True)
        """
        if self.augment:
            effects = [
                    ["lowpass", "-1", "300"], 
                    ["speed", f"{random.uniform(0.7, 1.3)}"],  # change speed
                  ]
            y, sr = torchaudio.sox_effects.apply_effects_tensor(
                y, sr, effects)
        #
            
        if self.augment:
            # augment sound in order to imitate the room change
            rir = self.rir[:, int(16000*1.1):int(16000*1.3)]
            rir = rir / torch.norm(rir, p=2)
            rir = torch.flip(rir, [1])
            y = torch.nn.functional.conv1d(y[None, ...], rir[None, ...])[0]
        
        """
        # here more augmentations could be used
        y = torchaudio.transforms.Resample(orig_freq=sr, new_freq=freq*1000)(y)

        """
        if self.augment:
            y = (0.5)*torch.randn(y.shape)
        """
        fixed_length = (freq*1000) * duration
        
        # returning result
        if y.shape[1] < fixed_length:
            y = torch.nn.functional.pad(
              y, (0, fixed_length - y.shape[1]))
        else:
            y = y[:, :fixed_length]
        result = {"target":self.targets.iloc[idx], "representation":y}
        return result

In [ ]:
train_ds = AudiosDataset(train_multiclass['filename'], train_multiclass['label'])
test_ds = AudiosDataset(test_multiclass['filename'], test_multiclass['label'])

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

batch_size = 2
num_workers = 10
loaders = {
    "train": DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        drop_last=True,
    ),
    "test": DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    ),
}

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
!pip install fairseq@git+https://github.com//pytorch/fairseq.git@f2146bdc7abf293186de9449bfa2272775e39e1d#egg=fairseq
!pip install sox

  Cloning https://github.com//pytorch/fairseq.git (to revision f2146bdc7abf293186de9449bfa2272775e39e1d) to /tmp/pip-install-_22tom43/fairseq_c5eb6cb8cc19490aa8a7be8f5781c01f
  Running command git clone -q https://github.com//pytorch/fairseq.git /tmp/pip-install-_22tom43/fairseq_c5eb6cb8cc19490aa8a7be8f5781c01f
  Running command git rev-parse -q --verify 'sha^f2146bdc7abf293186de9449bfa2272775e39e1d'
  Running command git fetch -q https://github.com//pytorch/fairseq.git f2146bdc7abf293186de9449bfa2272775e39e1d
  Running command git checkout -q f2146bdc7abf293186de9449bfa2272775e39e1d
  Running command git submodule update --init --recursive -q
  From https://github.com/ngoyal2707/Megatron-LM
   * branch            adb23324c222aad0aad89308e70302d996a5eaeb -> FETCH_HEAD
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 90 kB 4.2 MB

In [ ]:
from torch import nn
import torch.nn.functional as F

class ClassificationHead(nn.Module):
    """Head for aalbert classification task."""

    def __init__(self):
        super().__init__()
        self.dense = nn.Linear(768, 512)#config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(0.5)
        self.out_proj = nn.Linear(512, 3)#config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        x = features
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x

class Classifier(nn.Module):
    def __init__(self, extractor):
        super().__init__()
        self.extractor = extractor
        self.classifier = ClassificationHead()
    def forward(self, x):
        features = self.extractor(torch.squeeze(x))
        features = features[0]
        res = torch.mean(features, dim=1)
        logit = self.classifier(res)
        return logit

In [ ]:
from tqdm import tqdm
import math
from torch import autograd
import datetime


def train(model, opt, scheduler, loss_fn, epochs, data_tr, data_val, lr, last_epoch=0, data_augmentation = False, max_stable=15):
    best_val_loss = 1e9
    counter = 0
    augmented = '_augmented' if data_augmentation else ''
    nan=False
    for epoch in range(last_epoch, epochs):
        correct_predictions = 0
        count=0
        #tic = time()
        print('* Epoch %d/%d' % (epoch+1, epochs))

        avg_loss = 0
        model.train()  # train mode
        progress = tqdm(data_tr)
        for batch in progress:
            loss = 0
            # data to device
            X_batch, Y_batch = batch["representation"], batch["target"]
            # print(X_batch.shape)
            X_batch = X_batch.to(DEVICE)
            Y_batch = Y_batch.to(DEVICE)
            # set parameter gradients to zero
            opt.zero_grad()
            # forward
            Y_pred = model(X_batch)
            _, preds = torch.max(Y_pred, dim=1)
            preds = preds.to(DEVICE)
            Y_pred = Y_pred.to(DEVICE)

            loss = loss_fn(Y_pred, Y_batch)# forward-pass
            nan = math.isnan(loss)

            if nan:
              count=count+1
              nan=False
              continue
            correct_predictions += torch.sum(preds == Y_batch)
            progress.set_description(f'Loss: {loss:.2f} #nan->{count}')
            
            loss.backward()  # backward-pass
            opt.step()  # update weights
            if not scheduler is None:
                scheduler.step()
            # calculate loss to show the user
            avg_loss += loss
        avg_loss /= (len(data_tr)-(batch_size*count))
        avg_acc = correct_predictions.double() / ((len(data_tr)-count)*batch_size)
        
        print('loss: %f' % avg_loss)
        print('acc: %f' % avg_acc)
        # show intermediate results


        model.eval()  # testing mode
        val_loss = 0
        val_acc = 0
        print("start validation")
        count_loss_val=0
        progress_val = tqdm(data_val)
        correct_predictions_val = 0
        for v_b in progress_val:
            X_val, Y_val = v_b["representation"], v_b["target"]
            # X_val = X_val.to(DEVICE)#
            # Y_val = Y_val.to(DEVICE)#
            Y_hat = model(X_val.to(DEVICE)).detach().cpu()# detach and put into cpu
            
            _, preds_val = torch.max(Y_hat, dim=1)
            # preds_val = preds_val.to(DEVICE)#

            loss_val = loss_fn(Y_hat, Y_val)
            nan_loss = math.isnan(loss_val) #check loss==nan
            if nan_loss:
              count_loss_val = count_loss_val+1
              nan_loss = False
              continue
            #se trova un nan, salta tutto il batch
            correct_predictions_val += torch.sum(preds_val == Y_val)
            progress_val.set_description(f'Loss: {loss_val:.2f} #nan->{count_loss_val}')
            val_loss += loss_val

        val_loss /= (len(data_val)-(batch_size*count_loss_val))
        val_acc = correct_predictions_val.double() / ((len(data_val)-count_loss_val)*batch_size)
        print( f"validation loss: {val_loss}")
        print( f"validation accuracy: {val_acc}")
        if val_loss <= best_val_loss and val_loss > 0:
            counter = 0
            now = datetime.datetime.now()
            print(f'Save new model!- {now.day}/{now.month} h{now.hour+1}:{now.minute}')
            best_val_loss = val_loss
            torch.save(model.state_dict(), f'/content/drive/MyDrive/Tesi/DistilHubert/final_80-20_NO-BALANCED_bandPass2_dropout=0.5_PATHOLOGY-3-class_distilhubert_lr={lr}_batch={batch_size}_{freq}mhz_{duration}sec{augmented}.bin')
        else:
            counter += 1
        print("--------------------")
        if counter == max_stable:
            break

---------------------------

------------------------
Modello finale

In [ ]:
freq=16
duration=20
train_ds = AudiosDataset(train_multiclass['filename'], train_multiclass['label'])
test_ds = AudiosDataset(test_multiclass['filename'], test_multiclass['label'])
from torch.utils.data import TensorDataset, DataLoader

batch_size = 2
num_workers = 2
loaders = {
    "train": DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        drop_last=True,
    ),
    "test": DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    ),
}

In [ ]:
from transformers import AutoModel
DEVICE = 'cuda'
m = AutoModel.from_pretrained("ntu-spml/distilhubert")

Downloading:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/89.6M [00:00<?, ?B/s]

In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 20secondi tanh dropout=0.5 batch=8 distilhubert

* Epoch 1/100


Loss: 0.15 #nan->0: 100%|██████████| 368/368 [04:14<00:00,  1.45it/s]


loss: 0.463496
acc: 0.869565
start validation


Loss: 1.35 #nan->0: 100%|██████████| 92/92 [00:22<00:00,  4.12it/s]


validation loss: 0.3790121078491211
validation accuracy: 0.8804347826086957
Save new model!- 7/11 h16:59
--------------------
* Epoch 2/100


...

validation loss: 0.2901086211204529
validation accuracy: 0.9239130434782609
--------------------
* Epoch 25/100


Loss: 0.01 #nan->0: 100%|██████████| 368/368 [04:13<00:00,  1.45it/s]


loss: 0.045924
acc: 0.982337
start validation


Loss: 1.32 #nan->0: 100%|██████████| 92/92 [00:22<00:00,  4.10it/s]

validation loss: 0.19597657024860382
validation accuracy: 0.9347826086956522
--------------------
